<a href="https://colab.research.google.com/github/mpnsk/ivy_seminar/blob/main/my_ivy_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ivy
!pip install dm-haiku

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 4.8 MB/s eta 0:00:00


In [13]:
import ivy
my_backend = "torch"
# my_backend = "tensorflow"
# my_backend = "jax"

# ivy.set_default_device("cpu")

ivy.set_exception_trace_mode('full')

doCompile = True

ivy.set_backend(my_backend)
if my_backend == "jax":
  import jax
  jax.config.update('jax_enable_x64', True)


# as of the pytorch mnist example https://github.com/pytorch/examples/blob/main/mnist/main.py
class Net(ivy.Module):
    def __init__(self):
        self.conv1 = ivy.Conv2D(1, 32, [3, 3], 1, "VALID")
        self.conv2 = ivy.Conv2D(32, 64, [3, 3], 1, "VALID")
        self.conv1_drop = ivy.Dropout(0.25)
        self.conv2_drop = ivy.Dropout(0.5)
        self.fc1 = ivy.Linear(9216, 128)
        self.fc2 = ivy.Linear(128, 10)
        super().__init__()

    def _forward(self, x):
        x = self.conv1(x)
        x = ivy.relu(x)
        x = self.conv2(x)
        x = ivy.relu(x)
        x = ivy.max_pool2d(x, 2, 2, 'VALID')
        x = self.conv1_drop(x)
        x = ivy.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = ivy.relu(x)
        x = self.conv2_drop(x)
        x = self.fc2(x)
        x = ivy.softmax(x)
        return x


def loss_fn(v, x, y):
    entropy = ivy.sparse_cross_entropy(y, model(x))
    entropy = entropy.mean()
    return entropy


if doCompile:
  print("starting compilation")
  model = ivy.compile(Net(), to=my_backend, args=(ivy.random_normal(shape=(60,28,28,1)),))
else:
  model = Net()
print("done")

starting compilation
done


In [14]:
print('loading mnist')
from keras.datasets import mnist
(train_X_np, train_y_np), (test_X, test_y) = mnist.load_data()
del test_X # I'm not there yet, lol
del test_y

# no need to look at all 60,000 samples as of now
limit = 6400
train_X_np = train_X_np[:limit, :, :]
train_y_np = train_y_np[:limit]

# batch parameters
batch_size = 64
num_batches = limit // batch_size

print('starting to batch')
## convert to ivy
print(f"{type(train_X_np)=}")
train_X_ivy = ivy.array(train_X_np)
print(f"{type(train_X_ivy)=}")
print(f"{type(train_X_ivy.data)=}")

train_y_ivy = ivy.array(train_y_np)

# normalize from grey scale to 0.0-1.0
train_X_ivy = train_X_ivy / 255

# expand data from nhw to nhwc
train_X_ivy = ivy.expand_dims(train_X_ivy, axis=-1)

# turn big array into list of batched arrays
train_X_batches = [train_X_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
train_y_batches = [train_y_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
print('done')

loading mnist
starting to batch
type(train_X_np)=<class 'numpy.ndarray'>
type(train_X_ivy)=<class 'ivy.data_classes.array.array.Array'>
type(train_X_ivy.data)=<class 'torch.Tensor'>
done


In [ ]:
for idx in range(num_batches):
    loss, grads = ivy.execute_with_gradients(lambda v: loss_fn(v, train_X_batches[idx], train_y_batches[idx]), model.v)
    print(f'{idx=}, {loss=}')
    # print(f"{grads=}")
    if grads.all().cont_all_false():
        print("ERROR!")
        # print(f'{grads.conv1.w=}')
        # print(f'{grads.conv1.b=}')
        # print(f'{grads.conv2.w=}')
        # print(f'{grads.conv2.b=}')
        # print(f'{grads.fc1.w=}')
        # print(f'{grads.fc1.b=}')
        # print(f'{grads.fc2.w=}')
        # print(f'{grads.fc2.b=}')
        # break
    ivy.multiply(grads, 0.0005, out=grads)
    ivy.subtract(model.v, grads, out=model.v)
    # model.v = ivy.subtract(model.v, ivy.multiply(grads, 0.0005))



In [19]:
random_index = 3

x = train_X_ivy[random_index]
# expand hwc to nhwc
x = ivy.expand_dims(x, axis=0)
prediction = model(x)
print(f'{prediction=}')
print(f'{ivy.argmax(prediction)=}')
print(f'{train_y_ivy[random_index]=}')

prediction=tensor([[0.0157, 0.1683, 0.0212, 0.1423, 0.2341, 0.0110, 0.0387, 0.1549, 0.0814,
         0.1326]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
ivy.argmax(prediction)=ivy.array(4, dev=gpu:0)
train_y_ivy[random_index]=ivy.array(1, dev=gpu:0)
